# Part I: Construct The Model

### Just Have Fun Playing with It

In [4]:
## Please run in Glowscript: http://www.glowscript.org/#/user/b06902080/folder/Public/program/finalpublic/edit

from vpython import *
## define constants
mu0_over4pi = 1e-7
pi = acos(-1.0)
R = 0.15
I = 50
qe = 1.602e-19
mass_e = 9.11e-31
c = 3e8

In [5]:
enviro =canvas(width=600, height=400,title=  'click to shoot electron in')
apart = R
I1 = I2 = I
epos = vec(0,0,0)

## Build the coils

I_ring1 = extrusion(path=paths.circle(pos=vec(0.5*apart,0,0),radius=R*1.1,up=vec(1,0,0)),
                   texture=textures.metal, color = color.yellow,
                    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
                            [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
                            [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])

con_rod = cylinder(pos=vector(0.5*R, -R, 0), axis = vector(0, -0.2*R, 0), radius = 0.04*R, color = color.gray(0.8), shininess = 0.6)
base = box(pos=vector(0.5*R, -1.3*R, 0), size=vector(0.6*R, 0.3*R, 0.6*R), color = vector(1, 0.7, 0.2))

coil1 = compound([con_rod, base])
coil2 = coil1.clone(pos = vec(-coil1.pos.x, coil1.pos.y, 0))
I_ring2 = I_ring1.clone(pos = -I_ring1.pos)


## discretize the coils
segment_Y = []
segment_Z = []
for i in range(0, 73):
    segment_Y.append( R*cos(i*2*pi/72) )
    segment_Z.append( R*sin(i*2*pi/72) )

## For x-axis field plotting
x_field = graph(width=600,height=500, xtitle = 'x_position (*R)', ytitle = 'Magnetic field (T)', ymax = 4e-4, ymin = -4e-4)
gc_B1 = gcurve(graph=x_field,color=color.green)
gc_B2 = gcurve(graph=x_field,color=color.blue)
gc_B = gcurve(graph=x_field,color=color.magenta)

def x_plot(dis):
    gc_B1.delete()
    gc_B2.delete()
    gc_B.delete()
    segment_pos = []
    dLength = []
    
    for i in range(0, 72):
        segment_pos.append( 0.5*vec(0.5*dis, segment_Y[i], segment_Z[i]) + 
                            0.5*vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) )
        dLength.append( vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) - 
                        vec(0.5*dis, segment_Y[i], segment_Z[i]) )

    ## compute B-field along x-axis
    for i in range(-200, 201):
        now_pos = vec(i*(dis/200),0,0)
        B_now = vec(0,0,0)
        B1 = B2 = vec(0, 0, 0)
        
        for j in range (0,72):
            pos1 = segment_pos[j]
            pos2 = vec(-pos1.x, pos1.y, pos1.z)
            dB1 = mu0_over4pi*I1*cross(dLength[j], now_pos - pos1) / mag(now_pos - pos1)**3
            dB2 = mu0_over4pi*I2*cross(dLength[j], now_pos - pos2) / mag(now_pos - pos2)**3
            B1 += dB1
            B2 += dB2
            B_now += dB1+dB2

        gc_B.plot([now_pos.x/R, B_now.x])
        gc_B1.plot([now_pos.x/R, B1.x])
        gc_B2.plot([now_pos.x/R, B2.x])

## adjustible distance between coils
x_plot(apart)
enviro.caption = "Vary the distance between coils\n"
def setapart(s):
    wt.text = '{:1.2f}'.format(s.value)
    global apart
    apart = s.value*R
    coil1.pos.x = 0.5*apart
    coil2.pos.x = -0.5*apart
    I_ring1.pos.x = 0.5*apart
    I_ring2.pos.x = -0.5*apart
    x_plot(apart)
    
sl_ap = slider(min=0.5, max=3, value=1, length=220, bind=setapart, right=15)
wt = wtext(text='{:1.2f}'.format(sl_ap.value))
enviro.append_to_caption('* Radius of coil\n')

## adjust I1 current
enviro.append_to_caption('Vary the current on coil1 (right)\n')
def seti1(s):
    wti1.text = '{:1.1f}'.format(s.value)
    global I1
    I1 = s.value
    x_plot(apart)

sl_i1 = slider(min=-100, max=100, value=50, length=220, bind=seti1, right=15)
wti1 = wtext(text='{:1.1f}'.format(sl_i1.value))
enviro.append_to_caption(' Ampere\n')

## adjust I2 current
enviro.append_to_caption('Vary the current on coil2 (left)\n')
def seti2(s):
    wti2.text = '{:1.1f}'.format(s.value)
    global I2
    I2 = s.value
    x_plot(apart)

sl_i2 = slider(min=-100, max=100, value=50, length=220, bind=seti2, right=15)
wti2 = wtext(text='{:1.1f}'.format(sl_i2.value))
enviro.append_to_caption(' Ampere\n')

def B_calculate(pos):
    B_now = vec(0,0,0)
    for i in range (0,72):
        pos1 = ( 0.5*vec(0.5*apart, segment_Y[i], segment_Z[i]) + 
                 0.5*vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) )
        pos2 = vec(-pos1.x, pos1.y, pos1.z)
        dLength = ( vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) - 
                    vec(0.5*apart, segment_Y[i], segment_Z[i]) )
        dB1 = mu0_over4pi*I1*cross(dLength, pos - pos1) / mag(pos - pos1)**3
        dB2 = mu0_over4pi*I2*cross(dLength, pos - pos2) / mag(pos - pos2)**3
        B_now += dB1+dB2
    return B_now

## send an electron in
electron = sphere(pos=vec(0, 0, 0), radius = 0.03*R)
e_velo = vec(0, -7*1e-3*c, 0)
angle = 0
indicator = arrow(pos=electron.pos, axis=vec(0, -0.04*mag(e_velo)*R/(1e-3*c), 0), color = color.red)

## adjust e- initial position
Prestart = 1
enviro.append_to_caption('Set e- initial pos (x)\n')
def setx(s):
    wtx.text = '{:1.3f}'.format(s.value)
    global apart, epos, Prestart
    epos.x = s.value*apart
    if(Prestart): redraw(epos)

sl_x = slider(min=-0.5, max=0.5, value=0, length=220, bind=setx, right=15)
wtx = wtext(text='{:1.3f}'.format(sl_x.value))
enviro.append_to_caption('*(dis between coils) \n')

enviro.append_to_caption('Set e- initial pos (y)\n')
def sety(s):
    wty.text = '{:1.2f}'.format(s.value)
    global R, epos, Prestart
    epos.y = s.value*R
    if(Prestart): redraw(epos)

sl_y = slider(min=-1, max=1, value=0, length=220, bind=sety, right=15)
wty = wtext(text='{:1.2f}'.format(sl_y.value))
enviro.append_to_caption('*R \n')

enviro.append_to_caption('Set e- initial pos (z)\n')
def setz(s):
    wtz.text = '{:1.2f}'.format(s.value)
    global R, epos, Prestart
    epos.z = s.value*R
    if(Prestart): redraw(epos)

sl_z = slider(min=-1, max=1, value=0, length=220, bind=setz, right=15)
wtz = wtext(text='{:1.2f}'.format(sl_z.value))
enviro.append_to_caption('*R \n')

## Set initial speed of electron
enviro.append_to_caption('Set e- initial Speed\n')
def setv(s):
    wtv.text = '{:1.2f}'.format(s.value)
    global e_velo, Prestart, angle, indicator
    if(Prestart): 
        e_velo = vec(sin(angle/180*pi)*c*s.value*1e-3, -cos(angle/180*pi)*c*s.value*1e-3, 0)
        indicator.axis = mag(e_velo)*0.04*R/(1e-3*c)*norm(e_velo)

sl_vel = slider(min=3, max=10, value=7, length=220, bind=setv, right=15)
wtv = wtext(text='{:1.2f}'.format(sl_vel.value))
enviro.append_to_caption('*(10^-3 * speed of light) \n')

## Set injection angle
enviro.append_to_caption('Set e- Angle of injection\n')
def seta(s):
    wta.text = '{:1.2f}'.format(s.value)
    global e_velo, Prestart, angle, indicator
    if(Prestart):
        angle = s.value
        e_velo = mag(e_velo)*vec(sin(angle/180*pi), -cos(angle/180*pi), 0)
        indicator.axis = mag(e_velo)*0.04*R/(1e-3*c)*norm(e_velo)

sl_a = slider(min=-15, max=15, value=0, length=220, bind=seta, right=15)
wta = wtext(text='{:1.2f}'.format(sl_a.value))
enviro.append_to_caption('degree \n')

def redraw(ep):
    global electron
    electron.pos = ep
    indicator.pos = ep

## Electron motion
enviro.waitfor('click')
Prestart = 0
attach_trail(electron, retain=400)
indicator.visible = False

t = 0
dt = 1e-11
while(mag(electron.pos) <= 5*R):
    rate(100000)
    Fmag = -qe*cross(e_velo, B_calculate(electron.pos))
    e_velo += Fmag/mass_e*dt
    electron.pos += e_velo*dt
    t += dt

NameError: name 'canvas' is not defined

# Part II: Controlled Experiments

## Experiment 1: 
   ### Find Cyclotron Radius under various Initial Velocity
**Controlled Variables --**
   * `R: Radius of Coils = 0.15 m`
   * `Distance between coils = R` 
   * `I: Current on both coils: 50 A `
   * `Direction of e- injection: (0, -1, 0)`
   
   
**Independent Variable --**

   * Initial Velocity: 5 ~ 16 \* (10^-3\*speed of light), in 0.05 incremental steps
   
**Dependent Variable -- Cyclotron Radius**

In [ ]:
## Please run in Glowscript: http://www.glowscript.org/#/user/b06902080/folder/Public/program/Exp1/edit

from vpython import *
## define constants
mu0_over4pi = 1e-7
pi = acos(-1.0)
R = 0.15
I = 50
qe = 1.602e-19
mass_e = 9.11e-31
c = 3e8

enviro =canvas(width=600, height=400,title=  'click "start experiment"; then click on the scene to start')
apart = R
I1 = I2 = I
epos = vec(0,0,0)

## Build the coils
I_ring1 = extrusion(path=paths.circle(pos=vec(0.5*apart,0,0),radius=R*1.1,up=vec(1,0,0)),
                   texture=textures.metal, color = color.yellow,
                    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
                            [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
                            [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])

con_rod = cylinder(pos=vector(0.5*R, -R, 0), axis = vector(0, -0.2*R, 0), radius = 0.04*R, color = color.gray(0.8), shininess = 0.6)
base = box(pos=vector(0.5*R, -1.3*R, 0), size=vector(0.6*R, 0.3*R, 0.6*R), color = vector(1, 0.7, 0.2))

coil1 = compound([con_rod, base])
coil2 = coil1.clone(pos = vec(-coil1.pos.x, coil1.pos.y, 0))
I_ring2 = I_ring1.clone(pos = -I_ring1.pos)


## discretize the coils
segment_Y = []
segment_Z = []
for i in range(0, 73):
    segment_Y.append( R*cos(i*2*pi/72) )
    segment_Z.append( R*sin(i*2*pi/72) )

def B_calculate(pos):
    B_now = vec(0,0,0)
    for i in range (0,72):
        pos1 = ( 0.5*vec(0.5*apart, segment_Y[i], segment_Z[i]) + 
                 0.5*vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) )
        pos2 = vec(-pos1.x, pos1.y, pos1.z)
        dLength = ( vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) - 
                    vec(0.5*apart, segment_Y[i], segment_Z[i]) )
        dB1 = mu0_over4pi*I1*cross(dLength, pos - pos1) / mag(pos - pos1)**3
        dB2 = mu0_over4pi*I2*cross(dLength, pos - pos2) / mag(pos - pos2)**3
        B_now += dB1+dB2
    return B_now

## send an electron in
electron = sphere(pos=vec(0, 0, 0), radius = 0.03*R)
e_velo = vec(0, -7*1e-3*c, 0)
angle = 0
indicator = arrow(pos=electron.pos, axis=vec(0, -0.04*mag(e_velo)*R/(1e-3*c), 0), color = color.red)


#stop button
stop = 0
def setstop():
    global stop
    stop = 1
button(pos=enviro.title_anchor, bind=setstop, text='reset')

#experiment button
experiment = 0
def setexp():
    global experiment
    experiment = 1
button(pos=enviro.title_anchor, bind=setexp, text='start experiment')


#main
enviro.waitfor('click')
while (not experiment):
    stop = 0
    Prestart = 0
    indicator.visible = False

    t = 0
    dt = 1e-11
    while(mag(electron.pos) <= 5*R and stop != 1):
        rate(100000)
        Fmag = -qe*cross(e_velo, B_calculate(electron.pos))
        e_velo += Fmag/mass_e*dt
        electron.pos += e_velo*dt
        t += dt

    electron.pos=vec(0, 0, 0)
    e_velo = vec(0, -7*1e-3*c, 0)
    angle = 0
    trail.clear()
    ndicator.visible = True
    enviro.waitfor('click')

    if (experiment == 1):
        break



# Experiment 1
T = text(height=0.5*R, text='Experiment\nStart!', align='center', color=color.yellow, opacity=0.7)
for i in range(0, 70):
    rate(40)
    T.opacity = (70-i)/100.0
T.visible=False
indicator.visible = False
#trail.clear()

radgraph = graph(width = 600, height= 400, xtitle='Electron initial velocity *(10^-3*c)', ytitle='Cyclotron radius (m)')
real_curve = gcurve(graph = radgraph, color = color.orange)
theo_curve = gcurve(graph = radgraph, color = color.black)

## for theoretical plotting
for i in range(0, 221):
    velo = (5+0.05*i)*1e-3*c
    B = (8 * mu0_over4pi*4*pi * I) / (sqrt(125)*R)
    rad = (mass_e*velo)/(qe*B)
    theo_curve.plot([velo/(1e-3*c), rad])

## iterate to find correct radius
Zpos = []
lastpoint = 180
for i in range(0, 221):
    v = 5+0.05*i
    velo = vec(0, -(v*1e-3*c), 0)
    zpos = 0
    min = R
    for j in range(lastpoint, 1001):
        z = 0.001*j*R
        electron.pos = vec(0, 0, z)
        e_velo = velo

        t = 0
        dt = 1e-11
        while(abs(electron.pos.y) < R and abs(electron.pos.z) < R):
                rate(1000000)
                Fmag = -qe*cross(e_velo, B_calculate(electron.pos))
                e_velo += Fmag/mass_e*dt
                electron.pos += e_velo*dt
                t += dt
                if(abs(e_velo.y) >= 0.999*mag(e_velo) and e_velo.y > 0):  break
        
        if abs(electron.pos.z)-z <= 0.001*z:
            lastpoint = j
            zpos = z
            real_curve.plot([v, zpos])
            break

## Experiment 2(a):

### S.H.M. due to Injection with Horizontal(x-axis) velocity

#### Observation: Cyclotron motion resembles S.H.M. on the x-axis

**Controlled Variables --**
   * `R: Radius of Coils = 0.15 m`
   * `Distance between coils = R` 
   * `I: Current on both coils: 50 A `
   * `Initial Speed of Electron:` 
           i) 6 * (10^-3*speed of light)
          ii) 8 * (10^-3*speed of light)
   
   
**Independent Variable --**

   * Horizontal Angle of Declination: 0.1~10 Degrees, in 0.1 degree incremental steps
   
**Dependent Variables --**
   * S.H.M. Amplitude
   * S.H.M. Period
   * Angle at which S.H.M. collapses

In [ ]:
## Please Run in Glowscript: http://www.glowscript.org/#/user/b06902080/folder/Public/program/Exp2a/edit

from vpython import *

## define constants
mu0_over4pi = 1e-7
pi = acos(-1.0)
R = 0.15
I = 50
qe = 1.602e-19
mass_e = 9.11e-31
c = 3e8

enviro =canvas(width=600, height=400,title=  'click to shoot electron in')
e_velo = vec(0, -7*1e-3*c, 0)
apart = R
I1 = I2 = I
epos = vec(0,0,0)

## electron
electron = sphere(pos=vec(0, 0, 0), radius = 0.03*R)
angle = 0
Prestart = 1
I1_arr = arrow(pos=vector(0.5*apart+0.2*R, I1/1500, R), axis=vector(0, -I1/750, 0), color=color.green)
I2_arr = arrow(pos=vector(-0.5*apart-0.2*R, I2/1500, R), axis=vector(0, -I2/750, 0), color=color.cyan)
indicator = arrow(pos=electron.pos, axis=vec(0, -0.04*mag(e_velo)*R/(1e-3*c), 0), color = color.red)

## Functions
## discretize the coils
segment_Y = []
segment_Z = []
for i in range(0, 73):
    segment_Y.append( R*cos(i*2*pi/72) )
    segment_Z.append( R*sin(i*2*pi/72) )
    
def B_calculate(pos, choose):
    B_now = vec(0,0,0)
    for i in range (0,72):
        pos1 = ( 0.5*vec(0.5*apart, segment_Y[i], segment_Z[i]) + 
                 0.5*vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) )
        if(choose == 2): pos1.x = -pos1.x
        dLength = ( vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) - 
                    vec(0.5*apart, segment_Y[i], segment_Z[i]) )
        dB1 = mu0_over4pi*I1*cross(dLength, pos - pos1) / mag(pos - pos1)**3
        B_now += dB1
    return B_now

## Build the coils
I_ring1 = extrusion(path=paths.circle(pos=vec(0.5*apart,0,0),radius=R*1.1,up=vec(1,0,0)),
                   texture=textures.metal, color = color.yellow,
                    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
                            [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
                            [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])

con_rod = cylinder(pos=vector(0.5*apart, -R, 0), axis = vector(0, -0.2*R, 0), radius = 0.04*R, color = color.gray(0.8), shininess = 0.6)
base = box(pos=vector(0.5*apart, -1.3*R, 0), size=vector(0.6*R, 0.3*R, 0.6*R), color = vector(1, 0.7, 0.2))
coil1 = compound([con_rod, base])
coil2 = coil1.clone(pos = vec(-coil1.pos.x, coil1.pos.y, 0))
I_ring2 = I_ring1.clone(pos = -I_ring1.pos)

## Individual test plotting
xpos = graph(width=600, height=400, xtitle='t', ytitle='xpos')
xforce = graph(width=600, height=400, xtitle='xpos', ytitle='xforce')

posc = gcurve(graph=xpos, color=color.black)
Ftx = gcurve(graph=xforce, color=color.magenta)

## Exp 2a plotting
amplitude = graph(width=600, height=400, xtitle='Horizontal declination (degree)', ytitle='Amplitude (*R)')
period = graph(width=600, height=400, xtitle='Horizontal declination (degree)', ytitle='Period (s)')

ampdot = gdots(graph = amplitude, color=color.orange)
pedot = gdots(graph = period, color=color.blue)


'''
## For self-defined parameters
Prestart = 1

## Set initial speed of electron
enviro.append_to_caption('Set e- initial Speed\n')
def setv(s):
    wtv.text = '{:1.2f}'.format(s.value)
    global e_velo, Prestart, angle, indicator
    if(Prestart): 
        e_velo = vec(sin(angle/180*pi)*c*s.value*1e-3, -cos(angle/180*pi)*c*s.value*1e-3, 0)
        indicator.axis = mag(e_velo)*0.04*R/(1e-3*c)*norm(e_velo)

sl_vel = slider(min=3, max=10, value=7, length=220, bind=setv, right=15)
wtv = wtext(text='{:1.2f}'.format(sl_vel.value))
enviro.append_to_caption('*(10^-3 * speed of light) \n')

## Set injection angle
enviro.append_to_caption('Set e- Angle of injection\n')
def seta(s):
    wta.text = '{:1.3f}'.format(s.value)
    global e_velo, Prestart, angle, indicator
    if(Prestart):
        angle = s.value
        e_velo = mag(e_velo)*vec(sin(angle/180*pi), -cos(angle/180*pi), 0)
        indicator.axis = mag(e_velo)*0.04*R/(1e-3*c)*norm(e_velo)

sl_a = slider(min=-2, max=4, value=0, length=220, bind=seta, right=15)
wta = wtext(text='{:1.3f}'.format(sl_a.value))
enviro.append_to_caption(' degree \n')

enviro.waitfor('click')
Prestart = 0
## electron.pos = vec(0, 0, mag(e_velo)/(175.69*1e-3*c))
attach_trail(electron, retain=400) ## make trail using this
indicator.visible = False

t = 0
dt = 1e-11
while(mag(electron.pos) <= 5*R):
    rate(500000)
    Fmag1 = -qe*cross(e_velo, B_calculate(electron.pos, 1))
    Fmag2 = -qe*cross(e_velo, B_calculate(electron.pos, 2))
    Fmag = Fmag1+Fmag2
    e_velo += Fmag/mass_e*dt
    electron.pos += e_velo*dt
    Ftx.plot([electron.pos.x, Fmag.x])
    posc.plot([t, electron.pos.x])

    t += dt
'''


## Experiment 2a
enviro.waitfor('click')
velo = 3*1e-3*c
attach_trail(electron, retain=400)
indicator.visible = False
Failure = 0

for i in range(1, 101): ## declination from 0.1 to 10 degrees
    t = 0
    angle = i*0.1*pi/180
    amp_determined = 0
    dt = 1e-11
    e_velo = velo*vec(sin(angle), -cos(angle), 0)
    electron.pos = vec(0, 0, velo/(175.69*1e-3*c))
    
    Ftx.delete()
    posc.delete()
    
    while(True):
        if( electron.pos.x > 0.6*R ): ## Goes out of trap!!
            Failure = 1
            break
        
        rate(200000)
        Fmag1 = -qe*cross(e_velo, B_calculate(electron.pos, 1))
        Fmag2 = -qe*cross(e_velo, B_calculate(electron.pos, 2))
        Fmag = Fmag1+Fmag2
        e_velo += Fmag/mass_e*dt
        electron.pos += e_velo*dt
        
        ## Uncomment the following 2 lines to show force(x-axis) and x_postition
        ##Ftx.plot([electron.pos.x, Fmag.x])
        ##posc.plot([t, electron.pos.x])
        
        if( e_velo.x < 1e-8*mag(e_velo) and electron.pos.x > 1e-4 ):
            ampdot.plot([angle/pi*180, electron.pos.x/R])
            pedot.plot([angle/pi*180, 4*t])
            break
        t += dt
        
    if (Failure == 1) break


## Experiment 2(b):

### S.H.M. due to current change in I2

#### Observation: Cyclotron motion resembles S.H.M. on the x-axis

**Controlled Variables --**
   * `R: Radius of Coils = 0.15 m`
   * `Distance between coils = R` 
   * `I1: Current on coil 1: 50 A `
   * `Initial Speed of Electron:` 
           i) 6 * (10^-3*speed of light)
          ii) 8 * (10^-3*speed of light)
   * `Horizontal Angle = 0`
   
   
**Independent Variable --**

   * I2: Current on coil 2: -1.5 ~ 1.5 A, in 0.1 incremental steps
   
**Dependent Variables --**
   * S.H.M. Amplitude
   * S.H.M. Period

In [ ]:
## Please run in Glowscript: http://www.glowscript.org/#/user/b06902080/folder/Public/program/finalpublic/edit

from vpython import *

## define constants
mu0_over4pi = 1e-7
pi = acos(-1.0)
R = 0.15
I = 50
qe = 1.602e-19
mass_e = 9.11e-31
c = 3e8


enviro =canvas(width=600, height=400,title=  'click on the scene to start   ')
apart = R
I1 = I2 = I
epos = vec(0,0,0)

## Build the coils

I_ring1 = extrusion(path=paths.circle(pos=vec(0.5*apart,0,0),radius=R*1.1,up=vec(1,0,0)),
                   texture=textures.metal, color = color.yellow,
                    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
                            [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
                            [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])

con_rod = cylinder(pos=vector(0.5*R, -R, 0), axis = vector(0, -0.2*R, 0), radius = 0.04*R, color = color.gray(0.8), shininess = 0.6)
base = box(pos=vector(0.5*R, -1.3*R, 0), size=vector(0.6*R, 0.3*R, 0.6*R), color = vector(1, 0.7, 0.2))

coil1 = compound([con_rod, base])
coil2 = coil1.clone(pos = vec(-coil1.pos.x, coil1.pos.y, 0))
I_ring2 = I_ring1.clone(pos = -I_ring1.pos)

## arrows
pointer = arrow(pos=vector(R,R,R), axis=vector(0.7*R,0,0), shaftwidth=0.02*R, color=color.red)
pointer = arrow(pos=vector(R,R,R), axis=vector(0,0.7*R,0), shaftwidth=0.02*R, color=color.yellow)
pointer = arrow(pos=vector(R,R,R), axis=vector(0,0,0.7*R), shaftwidth=0.02*R, color=color.blue)
lx = text(height=0.1*R, pos=vector(1.5*R, 0.8*R, R), text='x', color=color.red)
ly = text(height=0.1*R, pos=vector(1.5*R, 0.6*R, R), text='y', color=color.yellow)
lz = text(height=0.1*R, pos=vector(1.5*R, 0.4*R, R), text='z', color=color.blue)

## discretize the coils
segment_Y = []
segment_Z = []
for i in range(0, 73):
    segment_Y.append( R*cos(i*2*pi/72) )
    segment_Z.append( R*sin(i*2*pi/72) )

## For x-axis field plotting
x_field = graph(width=600,height=500, xtitle = 'x_position (*R)', ytitle = 'Magnetic field (T)', ymax = 4e-4, ymin = -4e-4)
gc_B1 = gcurve(graph=x_field,color=color.green)
gc_B2 = gcurve(graph=x_field,color=color.blue)
gc_B = gcurve(graph=x_field,color=color.magenta)

def x_plot(dis):
    gc_B1.delete()
    gc_B2.delete()
    gc_B.delete()
    segment_pos = []
    dLength = []
    
    for i in range(0, 72):
        segment_pos.append( 0.5*vec(0.5*dis, segment_Y[i], segment_Z[i]) + 
                            0.5*vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) )
        dLength.append( vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) - 
                        vec(0.5*dis, segment_Y[i], segment_Z[i]) )

    ## compute B-field along x-axis
    for i in range(-200, 201):
        now_pos = vec(i*(dis/200),0,0)
        B_now = vec(0,0,0)
        B1 = B2 = vec(0, 0, 0)
        
        for j in range (0,72):
            pos1 = segment_pos[j]
            pos2 = vec(-pos1.x, pos1.y, pos1.z)
            dB1 = mu0_over4pi*I1*cross(dLength[j], now_pos - pos1) / mag(now_pos - pos1)**3
            dB2 = mu0_over4pi*I2*cross(dLength[j], now_pos - pos2) / mag(now_pos - pos2)**3
            B1 += dB1
            B2 += dB2
            B_now += dB1+dB2

        gc_B.plot([now_pos.x/R, B_now.x])
        gc_B1.plot([now_pos.x/R, B1.x])
        gc_B2.plot([now_pos.x/R, B2.x])

def B_calculate(pos):
    B_now = vec(0,0,0)
    for i in range (0,72):
        pos1 = ( 0.5*vec(0.5*apart, segment_Y[i], segment_Z[i]) + 
                 0.5*vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) )
        pos2 = vec(-pos1.x, pos1.y, pos1.z)
        dLength = ( vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) - 
                    vec(0.5*apart, segment_Y[i], segment_Z[i]) )
        dB1 = mu0_over4pi*I1*cross(dLength, pos - pos1) / mag(pos - pos1)**3
        dB2 = mu0_over4pi*I2*cross(dLength, pos - pos2) / mag(pos - pos2)**3
        B_now += dB1+dB2
    return B_now

## send an electron in
electron = sphere(pos=vec(0, 0, 0), radius = 0.03*R)
e_velo = vec(0, -7*1e-3*c, 0)
angle = 0



g2 = graph(width=600, height=150, title='<b>Amplitude to I2</b>', xtitle='<i>I2</i>', ytitle='<i>delta x</i>')
curve = gcurve(graph = g2, color = color.red)
curve2 = gcurve(graph = g2, color = color.black)


g3 = graph(width=600, height=150, title='<b>Period to I2</b>', xtitle='<i>I2</i>', ytitle='<i>Period</i>')
curve3 = gcurve(graph = g3, color = color.red)




enviro.waitfor('click')
trail = attach_trail(electron, retain=400)

#Experiment 2
T = text(height=0.5*R, text='Experiment 2\nStart!', align='center', color=color.yellow, opacity=0.7)
for i in range(0, 70):
    rate(40)
    T.opacity = (70-i)/100.0
T.visible=False


i2 = []
p = []
dx_max = []
dx_min = []
for i in range(-15, 15):
    g1 = graph(width=600, height=150, title='<b>dx to t</b>', xtitle='<i>t</i>', ytitle='<i>delta x</i>')
    curve0 = gcurve(graph = g1, color = color.red)
    
    gf = graph(width=600, height=150, title='<b>F to dx</b>', xtitle='<i>dx</i>', ytitle='<i>F</i>')
    curvef = gcurve(graph = gf, color = color.red)
    
    trail.clear()
    e_velo = vec(0, -7*1e-3*c, 0)
    electron.pos=vec(0, 0, mag(e_velo)/(175.69*1e-3*c))
    
    I2 = 50+i/10.0
    t = 0
    dt = 1e-11
    cnt = 0
    max = 0
    min = R
    fail = False
    while(mag(electron.pos) <= R and cnt < 2 and i != 0):
        rate(100000)
        Fmag = -qe*cross(e_velo, B_calculate(electron.pos))
        e_velo += Fmag/mass_e*dt
        if ((electron.pos+e_velo*dt).x * electron.pos.x < 0):
            cnt = cnt+1
        electron.pos += e_velo*dt
        t += dt
        if (mag(electron.pos) > R):
            fail = True
        if (electron.pos.x > max):
            max = electron.pos.x
        elif (electron.pos.x < min):
            min = electron.pos.x
        curve0.plot([t, electron.pos.x])
        curvef.plot([electron.pos.x, Fmag.x])
    
    if (not fail and i!=0):
        curve.plot([I2, max])
        curve2.plot([I2, min])
        curve3.plot([I2, t])
        dx_max.append(max)
        dx_min.append(min)
        p.append(t)
        i2.append(I2)
    else:
        curve0.delete()
        curvef.delete()
        g1.delete()
        gf.delete()
        
print ("When value of I2 is between %d and %d, it does not crash") % (i2[0], i2[-1])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>